In [72]:
import pandas as pd
import numpy as np
import json
from os import listdir
from os.path import isfile, join

In [73]:
def load_files(filedir):
  with open(filedir) as f:
    file = json.loads(f.read())
    return file

# Pre-Processing

!git clone https://github.com/marcoramilli/MalwareTrainingSets/

In [74]:
mypath='/Users/fernanda/Documents/Hands On Machine Learning for Cybersecurity/Cybersecurity-MachineLearning/MalwareTrainingSets/trainingSets'
malwaretype=["APT1","Crypto","Locker","Zeus"]

apt1_files=[join(join(mypath,malwaretype[0]),f) for f in listdir(join(mypath,malwaretype[0])) if isfile(join(join(mypath,malwaretype[0]), f))]
crypto_files=[join(join(mypath,malwaretype[1]),f) for f in listdir(join(mypath,malwaretype[1])) if isfile(join(join(mypath,malwaretype[1]), f))]
locker_files=[join(join(mypath,malwaretype[2]),f) for f in listdir(join(mypath,malwaretype[2])) if isfile(join(join(mypath,malwaretype[2]), f))]
zeus_files=[join(join(mypath,malwaretype[3]),f) for f in listdir(join(mypath,malwaretype[3])) if isfile(join(join(mypath,malwaretype[3]), f))]

apt1_files = [load_files(filedir=name) for name in apt1_files]
crypto_files = [load_files(filedir=name) for name in crypto_files]
locker_files = [load_files(filedir=name) for name in locker_files]
zeus_files = [load_files(filedir=name) for name in zeus_files]

In [75]:
# archive structure:
print(apt1_files[0].keys())
print(crypto_files[0].keys())
print(locker_files[0].keys())
print(zeus_files[0].keys())
print("\n")
print('properties apt1 file 0:',len(apt1_files[0]['properties'].keys()))
print('properties crypto file 0:',len(crypto_files[0]['properties'].keys()))
print('properties locker file 0:',len(locker_files[0]['properties'].keys()))
print('properties zeus file 0:',len(zeus_files[0]['properties'].keys()))
#each kind of malware has different properties in its files

dict_keys(['entityId', 'entityType', 'event', 'eventTime', 'properties'])
dict_keys(['entityId', 'entityType', 'event', 'eventTime', 'properties'])
dict_keys(['entityId', 'entityType', 'event', 'eventTime', 'properties'])
dict_keys(['entityId', 'entityType', 'event', 'eventTime', 'properties'])


properties apt1 file 0: 22
properties crypto file 0: 18
properties locker file 0: 17
properties zeus file 0: 34


In [76]:
apt1_prop = []
crypto_prop = []
locker_prop = []
zeus_prop = []

for file in apt1_files:
  for prop in file['properties']:
    if (prop not in apt1_prop):
        apt1_prop.append(prop)
for file in crypto_files:
  for prop in file['properties']:
    if (prop not in crypto_prop):
        crypto_prop.append(prop)
for file in locker_files:
  for prop in file['properties']:
    if (prop not in locker_prop):
        locker_prop.append(prop)
for file in zeus_files:
  for prop in file['properties']:
    if (prop not in zeus_prop):
        zeus_prop.append(prop)

#check common properties
props = []
for prop in apt1_prop:
  if (prop in crypto_prop and prop in locker_prop and zeus_prop):
    props.append(prop)

print('common properties:',len(props))

common properties: 52


In [77]:
## create dataframe
df=pd.DataFrame(columns=props)
        
hexa=apt1_files[0]['properties'].get(props[0])

def append_df(file,properties):
    line=[]
    for i in range(0,len(properties)):
        try: line.append(file['properties'].get(props[i]))
        except: line.append()
    return line

for f in apt1_files:
    df.loc[len(df)]=append_df(f,props)
for f in crypto_files:
    df.loc[len(df)]=append_df(f,props)
for f in locker_files:
    df.loc[len(df)]=append_df(f,props)
for f in zeus_files:
    df.loc[len(df)]=append_df(f,props)

In [78]:
df.describe()

,file_access,pe_imports,reg_read,sig_antisandbox_sleep,reg_write,pe_sec_character,label,api_resolv,pe_sec_entropy,pe_sec_name,...,sig_packer_entropy,sig_exec_crash,sig_stealth_network,sig_network_cnc_http,sig_origin_langid,sig_antiav_servicestop,sig_process_interest,sig_modifies_certs,sig_encrypted_ioc,sig_injection_createremotethread
count,3696,4557,3511,263,1529,4750,4762,4028,4750,4750,...,2934,6,70,42,68,18,216,47,173,308
unique,1908,1212,861,209,501,200,4,1321,1850,351,...,831,1,1,22,1,7,54,1,71,160
top,bda9f42e 3c6b9e80 06416233,69fe6141 9346f95b cec36c4f 37aa7177 dfd10ba1 6...,1fe39017 df16f4d5,9f4c2c60,75ff7b0e a72671f7,d78064fc eb95869a ad8225f3,Crypto,e6e88ee5 8c70d8b3 090a162f 2572f7e8 4b03b39a a...,63790b7d 566d705e 247ea3d9 6915b647,c57b5e67 46040620 1f9f1073,...,5617b9b7,,,bb03f7af 543b370c b26cf15b,,10fc4023 e788ca34 e557c1c1 cc411e6c 7a7e8242,cde09bcd,,23037168,69eb7452
freq,96,106,1140,4,182,1089,2020,240,78,1114,...,78,6,70,6,68,6,28,47,21,2


In [82]:
#drop columns with more than 40% of values=null
dfsomanulo=df.isna().sum().sort_values(ascending=False)
l,c=df.shape
drp=[]
for i in range(0,len(dfsomanulo)):
    if dfsomanulo[i]/l>0.4:
        drp.append(dfsomanulo.index[i])
df=df.drop(drp,axis=1)
df.describe()

,file_access,pe_imports,reg_read,pe_sec_character,label,api_resolv,pe_sec_entropy,pe_sec_name,sig_antivirus_virustotal,file_read,reg_access,str,sig_injection_rwx,sig_packer_entropy
count,3696,4557,3511,4750,4762,4028,4750,4750,3595,3644,3902,4759,3532,2934
unique,1908,1212,861,200,4,1321,1850,351,2423,1739,1147,1994,1,831
top,bda9f42e 3c6b9e80 06416233,69fe6141 9346f95b cec36c4f 37aa7177 dfd10ba1 6...,1fe39017 df16f4d5,d78064fc eb95869a ad8225f3,Crypto,e6e88ee5 8c70d8b3 090a162f 2572f7e8 4b03b39a a...,63790b7d 566d705e 247ea3d9 6915b647,c57b5e67 46040620 1f9f1073,a8608aa7 a4d696b9 f42f8a61 f42f8a61 47f83a65 5...,bda9f42e 3c6b9e80 06416233,efc50fc9 1fe39017 1fe39017 df16f4d5,916e7571 ec73657d e4787bb1 1f9f1073 86f14530 1...,,5617b9b7
freq,96,106,1140,1089,2020,240,78,1114,2,96,1022,78,3532,78
